### Check `kcorrect` version

In [1]:
import kcorrect
print(f"kcorrect version: {kcorrect.__version__}")

kcorrect version: 5.1.8


### Check filter response files (filter names)

In [5]:
import kcorrect
import os

kcorrect_path = os.path.dirname(kcorrect.__file__)
data_path = os.path.join(kcorrect_path, 'data')
responses_path = os.path.join(data_path, 'responses')

# Check files in the responses directory
if os.path.exists(responses_path):
    response_files = os.listdir(responses_path)
    for i in ['process', 'test', 'README', 'header']:
        response_files = [f for f in response_files if i not in f]
    print(f"Number of files in responses directory: {len(response_files)}")
    print("\nAll filter response files:")
    
    # Sort alphabetically and display
    for i, f in enumerate(sorted(response_files), 1):
        # Remove file extension to get filter name
        filter_name = os.path.splitext(f)[0]
        print(f"{i:3d}. {filter_name}")
    
    print(f"\nTotal filters found: {len(response_files)}")
else:
    print(f"Responses directory does not exist: {responses_path}")

Number of files in responses directory: 354

All filter response files:
  1. FORS2_R_ccd_atm
  2. FORS_B_ccd_atm
  3. FORS_I_ccd_atm
  4. FORS_V_ccd_atm
  5. SOFI_J_atm
  6. SOFI_Ks_atm
  7. acs
  8. alhambra_768
  9. alhambra_799
 10. alhambra_830
 11. alhambra_861
 12. alhambra_892
 13. alhambra_923
 14. alhambra_954
 15. alhambra_H
 16. alhambra_J
 17. alhambra_KS
 18. alhambra_f365w
 19. alhambra_f396w
 20. alhambra_f427w
 21. alhambra_f458w
 22. alhambra_f489w
 23. alhambra_f520w
 24. alhambra_f551w
 25. alhambra_f582w
 26. alhambra_f613w
 27. alhambra_f644w
 28. alhambra_f675w
 29. alhambra_f706w
 30. alhambra_f737w
 31. alhambra_f768w
 32. alhambra_f799w
 33. alhambra_f830w
 34. alhambra_f861w
 35. alhambra_f892w
 36. alhambra_f923w
 37. alhambra_f954w
 38. bass_g
 39. bass_r
 40. bessell
 41. bessell_B
 42. bessell_I
 43. bessell_R
 44. bessell_U
 45. bessell_V
 46. blah
 47. bok_90prime_z
 48. capak_cfht_megaprime_sagem_g
 49. capak_cfht_megaprime_sagem_i
 50. capak_cfht_megap

### Preprocessing the templates

```python
import kcorrect

# Include all the filters in your data.
responses = ['FILTER1', 'FILTER2', 'FILTER3']

# kcorrect template
templates = kcorrect.template.Template(filename='YOUR TEMPLATE FILE')

# The minimum redshift should below 0, for deriving the fitted physical parameters later.
kc = kcorrect.kcorrect.Kcorrect(responses=responses, templates=templates, redshift_range=[-0.01, 0.5], nredshift=10)

kc.tofits('data/kcorrect_templates_test.fits')
```
